In [ ]:
# Importing Useful Libraries
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras import optimizers
from keras.models import load_model
from tensorflow.keras.preprocessing import image
import os
from os import listdir
import os
import random
import scipy
import pylab as pl
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
Train_data_dir=r'C:\Users\User\Downloads\Images'
validation_data_dir=r'C:\Users\User\Downloads\Images'

In [ ]:
# Seeing the Number of classes in the training folder
file = os.listdir(r"C:\Users\User\Downloads\Images")
file

### Loading Some Scrapped Image Samples

In [ ]:
# Sample images which I have scrapped 
import matplotlib.image as mpimg
Jeans_train=r"C:\Users\User\Downloads\Images\Jeans(men)"
Saree_train=r"C:\Users\User\Downloads\Images\saree"
Trouser_train=r"C:\Users\User\Downloads\Images\Trousers(Men)"


Dir_train=[Jeans_train, Saree_train, Trouser_train]
for dirs in Dir_train:
    k=listdir(dirs)
    for i in k[:2]:
        img=mpimg.imread('{}/{}'.format(dirs,i))
        plt.imshow(img)
        plt.axis('off')
        plt.show()

In [ ]:
# Defining Dimensions for the image to be input and then loading the images
input_shape=(576,576,3)
img_width=576
img_height=576
nb_train_samples=192
nb_validation_samples=48
batch_size=8
epoch=150

### Preparing Data for Training

In [ ]:
# Training Data Generator( Data Augmentation on Training Images)

Train_generator_augmented=ImageDataGenerator(rescale=1./255,
                                             zoom_range=0.2,
                                             rotation_range=30,
                                             horizontal_flip=True)
Train_generator=Train_generator_augmented.flow_from_directory(Train_data_dir,
                                                              target_size=(img_width,img_height),
                                                              batch_size=batch_size, 
                                                              class_mode='categorical')

# Validation Data Generator
Data_gen=ImageDataGenerator(rescale=1./255)
validation_generator=Data_gen.flow_from_directory(validation_data_dir,
                                                  target_size=(img_width,img_height),
                                                  batch_size=batch_size, 
                                                  class_mode='categorical')

### Defining the Model

In [ ]:
# Creating the model
model=Sequential()

# First convolution layer
model.add(Conv2D(32,(3,3),input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Second convolution layer
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Third convolution layer
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Fourth convolution layer
model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

print(model.summary())

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

### Defining Early stopping and Model check point

In [ ]:
# Defining Early stopping and Model check point
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

ES = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
MC = ModelCheckpoint('Image_Classification.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

### Fitting the data into the model

In [ ]:
# Fitting the Training Data
history = model.fit_generator(
    Train_generator, 
    epochs=epoch,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size,
    steps_per_epoch=nb_train_samples//batch_size,
    callbacks=[ES,MC])

In [ ]:
# Model Evaluation
evl=model.evaluate(validation_generator,steps=1)
print("Test Loss",evl[0])
print("Test Accuracy",evl[1])

### On 43 Epoch itself the val_accuracy is: 93.76% and val_loss: 43.95%,thus saving the best model

### saving the best model

In [ ]:
# Saving the best model where val_accuracy is maximum.
model.save('Image_Classification.h5')

In [ ]:
# 150 Epochs Summary
pd.DataFrame(history.history)

### Visualizing Training

In [ ]:
# Visualizing  Training
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(30, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, epoch, 1),)
ax1.set_xlabel('Epochs' ,fontsize=50)
ax1.set_ylabel('Loss' ,fontsize=50)
ax1.legend(loc='best', shadow=True)

ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xlabel('Epochs' ,fontsize=50)
ax2.set_ylabel('Accuracy' ,fontsize=50)
ax2.set_xticks(np.arange(1, epoch, 1))

ax2.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

### As we can see that accuracy is increasing and losses are decreasing as number of epochs are increasing.

In [ ]:
# As we can see that there are 3 classes for the 144 test images
print(validation_generator.classes)

In [ ]:
# Lets load our model

saved_model = load_model('Image_Classification.h5')

### Predicting Images

In [ ]:
 #Predicted samples of the test images
test_jeans=r"C:\Users\User\Downloads\Images\Jeans(men)"
test_Saree=r"C:\Users\User\Downloads\Images\saree"
test_trouser=r"C:\Users\User\Downloads\Images\Trousers(Men)"


Predicted_class=[]
classes=[test_jeans,test_Saree,test_trouser]
for test_dir in classes:
    for i in listdir(test_dir):
        print("Input Image is:",i)
        img= image.load_img('{}/{}'.format(test_dir,i))                         
        test_image = image.load_img('{}/{}'.format(test_dir,i),target_size=(576, 576))
        test_image = image.img_to_array(test_image)
        plt.imshow(img)
        plt.axis('off')
        plt.show()
        test_image = np.expand_dims(test_image, axis=0)
        result = saved_model.predict(test_image)
        final=np.argmax(result, axis=1)[0]
        if final==0:
            print("Predicted Label is: jeans\n")
            Predicted_class.append("Jeans men")
        elif final==1:
            print("Predicted Label is: sarees\n")
            Predicted_class.append("Sarees")
        elif final==2:
            print("Predicted Label is: trouser\n")
            Predicted_class.append("trouser men")

### DataFrame to show the Actual and Predicted Label

In [ ]:
res = pd.DataFrame()
res['Predicted_Label']=Predicted_class
res['Actual_Label']=validation_generator.classes
res["Actual_Label"] = res["Actual_Label"].replace({0: 'jeans men', 1: 'Sarees',2:'Trouser men'})
res